# **Sales Pirce Study Notebook**

# Objectives

*   Answer business requirement 1: 
    * 1 - The client is interested in discovering how the house attributes correlate with the sale price. Therefore, the client expects data visualisations of the correlated variables against the sale price to show that.
## Inputs

* outputs/datasets/collection/house_prices_records.csv

## Outputs

* generate code that answers business requirement 1 and can be used to build the Streamlit App






---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [11]:
import os
current_dir = os.getcwd()
current_dir

'/workspace'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [12]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [13]:
current_dir = os.getcwd()
current_dir

'/'

# Load Data

Section 1 content

In [14]:
import pandas as pd
df = pd.read_csv("/workspace/milestone-project-heritage-housing-issues/outputs/datasets/collection/house_prices_records.csv")

    
df.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500


---

# Data Exploration

We are looking deeper into the data set to check variable type and distribution, missing levels and what these variables mean in a business context

In [15]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()


ModuleNotFoundError: No module named 'ipywidgets'

# Corroleaton study

---

Now we will do a test with spearmanns and perrsons to investigate corriltaion between the varriables and sales price

In [19]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=df.columns[df.dtypes=='object'].to_list(), drop_last=False)
df_ohe = encoder.fit_transform(df.dropna())
print(df_ohe.shape)
df_ohe.head(3)

(17, 38)


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtFinSF1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageYrBlt,GrLivArea,LotArea,...,BsmtFinType1_Rec,BsmtFinType1_LwQ,BsmtFinType1_Unf,BsmtFinType1_None,GarageFinish_RFn,GarageFinish_Unf,GarageFinish_Fin,KitchenQual_Gd,KitchenQual_TA,KitchenQual_Ex
0,856,854.0,3.0,706,150,0.0,548,2003.0,1710,8450,...,0,0,0,0,1,0,0,1,0,0
228,912,0.0,3.0,842,70,0.0,336,1974.0,912,8521,...,0,0,0,0,0,1,0,0,1,0
279,1156,866.0,4.0,392,768,0.0,505,1977.0,2022,10005,...,0,0,0,0,0,0,1,0,1,0


In [20]:
corr_spearman = df_ohe.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(15)
corr_spearman

OverallQual         0.882896
GarageFinish_Unf   -0.769800
TotalBsmtSF         0.713235
GarageFinish_Fin    0.678401
BsmtFinType1_GLQ    0.658808
YearRemodAdd        0.646592
YearBuilt           0.640446
OpenPorchSF         0.619776
GarageYrBlt         0.609816
GrLivArea           0.598039
1stFlrSF            0.534314
GarageArea          0.526961
LotFrontage         0.489571
KitchenQual_TA     -0.481125
BsmtFinType1_ALQ   -0.447214
Name: SalePrice, dtype: float64

In [21]:
corr_pearson = df_ohe.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(15)
corr_pearson

KitchenQual_Ex      0.961662
GrLivArea           0.923992
MasVnrArea          0.897842
1stFlrSF            0.837574
OverallQual         0.812240
LotArea             0.779845
TotalBsmtSF         0.777774
2ndFlrSF            0.758421
GarageArea          0.706019
BsmtExposure_Gd     0.666548
BsmtFinSF1          0.644904
BsmtFinType1_GLQ    0.533496
GarageFinish_Fin    0.493240
GarageFinish_Unf   -0.446748
BedroomAbvGr        0.426873
Name: SalePrice, dtype: float64

In both correlation tests, there are strong corrilations in the top 5 for each but peerson shows stong corrilation overall in top 10.

We will consider the top five correlation levels at `df_ohe` and will study the associated variables at `df`

In [27]:
top_n = 5
vars_to_study = corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list()
vars_to_study

['KitchenQual_Ex',
 'GrLivArea',
 'MasVnrArea',
 '1stFlrSF',
 'OverallQual',
 'OverallQual',
 'GarageFinish_Unf',
 'TotalBsmtSF',
 'GarageFinish_Fin',
 'BsmtFinType1_GLQ']

In [31]:
vars_to_study = ['1stFlrSF', 'BsmtFinType1', 'GarageFinish', 'GrLivArea', 'KitchenQual', 'MasVnrArea', 'OverallQual',
        'TotalBsmtSF']

# EDA on selected variables

In [32]:
df_eda = df.filter(vars_to_study + ['SalePrice'])
df_eda.head(3)

,1stFlrSF,BsmtFinType1,GarageFinish,GrLivArea,KitchenQual,MasVnrArea,OverallQual,TotalBsmtSF,SalePrice
0,856,GLQ,RFn,1710,Gd,196.0,7,856,208500
1,1262,ALQ,RFn,1262,TA,0.0,6,1262,181500
2,920,GLQ,RFn,1786,Gd,162.0,7,920,223500


## Variables Distribution by Churn

We plot the distribution (numerical and categorical) coloured by Sales price

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
target_var = 'SalePrice'

def plot_categorical(df, col, target_var):

    plt.figure(figsize=(12, 5))
    sns.countplot(data=df, x=col, hue=target_var, order=df[col].value_counts().index)
    plt.xticks(rotation=90)
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()


def plot_numerical(df, col, target_var):
    plt.figure(figsize=(8, 5))
    sns.histplot(data=df, x=col, hue=target_var, kde=True, element="step")
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()



for col in vars_to_study:
    if df_eda[col].dtype == 'object':
        plot_categorical(df_eda, col, target_var)
        print("\n\n")
    else:
        plot_numerical(df_eda, col, target_var)
        print("\n\n")


NOTE

* You may add as many sections as you want, as long as they support your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
# import os
# try:
#   # create here your folder
#   # os.makedirs(name='')
# except Exception as e:
#   print(e)
